In [1]:
from glob import glob
from framed import load_cbmodel
import pandas as pd
import numpy as np
from framed.io.sbml import sanitize_id
import reframed
from reframed.cobra.ensemble import load_ensemble
import cobra
from cobra.io import read_sbml_model
import pandas as pd
import re

In [2]:
NeuroMetab=['glu-L[c]', 'glu-L[e]', 'glu-D[c]', 'trp-L[c]', 'trp-L[e]', 'taur[e]', 'taur[c]', 'but[c]', 'ppa[c]', 'ppa[e]', 'ac[c]', 'ac[e]','inost[c]', 
                 'mqn8[c]', 'mqn8[e]','mqn9[e]','mqn9[c]','mqn10[c]','mqn10[e]','mqn11[c]','mqn11[e]','mqn7[c]','mqn7[e]'
                , 'quln[c]', 'amet[c]']
print(len(NeuroMetab))
Curated_NeuroMetabs=pd.DataFrame(NeuroMetab, columns=['BiGGIDnoM'])
# Curated_NeuroMetabs.to_excel('curated_neuroMetabs.xlsx')

25


In [3]:
M9Series=pd.read_excel('BthetaCuratedMinimalMedium.xlsx', index_col=0)
M9List=M9Series.index.tolist()
Media=pd.read_excel('carvemeMedia.xlsx')
M9CarvemeMedium=Media['M9'].tolist()
CarvemeM9Metabs=list(set(M9CarvemeMedium) - set(M9List))
M9Medium=M9List+CarvemeM9Metabs
LBmediumList=Media['LB'].tolist()
LBmedium=LBmediumList+M9Medium
MediumList=[LBmedium]
for medium in MediumList:
    Model=read_sbml_model('Btheta_curated.xml')  
    Model.optimize()
    ModelMedium=[v for v in Model.medium.keys()] # has all Exchange reactions active
    ModelMedium.remove("EX_glc(e)")
    Model.reactions.get_by_id('EX_o2(e)').bounds = (0.0, 1000.0)        
    for mediumRx in ModelMedium: #go through each reaction in medium
        if mediumRx not in medium: #put the rxs that are not in M9 composition as inactive so that we have M9 medium
            Model.reactions.get_by_id(mediumRx).bounds = (0.0, 1000.0) #put the flux as INACTIVE
    for Ex_rx in Model.medium.keys():
        Model.reactions.get_by_id(Ex_rx).lower_bound = -10
        Model.reactions.get_by_id(Ex_rx).upper_bound = 1000
    Model.optimize()
    ModelMetabs=[]
    ReactionsIdData=[]
    ReactionsIdDataIndexNames=[]
    TransportersINneuroRXs=[]
    TransptGPRList=[]
    UMetabList=[]

    for NeuroactiveMetab in NeuroMetab:
#         print(NeuroactiveMetab)

        reactionsID=[]
        genesList=[]

        producingDF=Model.metabolites.get_by_id(NeuroactiveMetab).summary().producing_flux
        consumingDF=Model.metabolites.get_by_id(NeuroactiveMetab).summary().consuming_flux

        if producingDF.empty==False:
            slicedProducingDF=producingDF[['reaction']]
            listOfProdStrings = ['producing'] * slicedProducingDF.shape[0]
            slicedProducingDF.insert(1, "RxType", listOfProdStrings, True)

        if consumingDF.empty==False:
            slicedConsumingDF=consumingDF[['reaction']]
            listOfConsumStrings = ['consuming'] * slicedConsumingDF.shape[0]
            slicedConsumingDF.insert(1, "RxType", listOfConsumStrings, True)

            AllRXsDF = pd.concat([slicedProducingDF, slicedConsumingDF])

            for index, rows in AllRXsDF.iterrows():    # Put REACTIONids in a list so that we can later loop over it
                RowIndex = str(NeuroactiveMetab)
                Row =[rows.reaction, RowIndex, rows.RxType]
                ReactionsIdData.append(Row)

ReactionsIdDf=pd.DataFrame(ReactionsIdData, columns=['reaction',  'metabolite', 'RxType'])
ReactionsIdDf=ReactionsIdDf.sort_values('reaction')
print('dataframe with reactions \n:', ReactionsIdDf)

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
dataframe with reactions 
:        reaction metabolite     RxType
29   26DAPLLATi   glu-L[c]  consuming
30         ACGS   glu-L[c]  consuming
70         ACKr      ac[c]  producing
71        ACODA      ac[c]  producing
0         ACOTA   glu-L[c]  producing
..          ...        ...        ...
27        TYRTA   glu-L[c]  producing
47       UAMAGS   glu-D[c]  consuming
79    UHGADA_HP      ac[c]  producing
128      URFGTT    amet[c]  consuming
28        VALTA   glu-L[c]  producing

[129 rows x 3 columns]


In [6]:
NeuroUniversalRxs=ReactionsIdDf.reaction.unique()
NeuroUniversalRxsDF = pd.DataFrame(NeuroUniversalRxs, columns=['ReactionID'])

In [7]:
# NeuroUniversalRxsDF.to_excel('curated_neuroRxs.xlsx')